In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# === Load Temporal and Spatial Data ===
temporal_df = pd.read_csv("humidity_temporal.csv")
spatial_df = pd.read_csv("humidity_spatial.csv")

# === Merge on Segment_ID ===
merged_df = pd.merge(temporal_df, spatial_df, on='Segment_ID')

# === Label Encoding ===
label_encoder = LabelEncoder()
merged_df['Label'] = label_encoder.fit_transform(merged_df['Defect_Label'])

# === Feature Selection ===
feature_cols = [col for col in temporal_df.columns if col.startswith('t')]
X = merged_df[feature_cols].values
y = merged_df['Label'].values

# === Normalize Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Reshape for LSTM: (samples, timesteps, features) ===
# Here: 100 time steps, 3 features (original humidity + 2 moving averages)
X_lstm = X_scaled.reshape((X.shape[0], 100, 3))

# === One-Hot Encode Labels ===
y_cat = to_categorical(y)

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_cat, test_size=0.2, random_state=42, stratify=y)

# === LSTM Model ===
model = Sequential()
model.add(LSTM(64, input_shape=(100, 3), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_cat.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# === Train ===
model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.1, verbose=1)

# === Evaluate ===
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\n✅ Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


c:\Users\Haard\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,620 (76.64 KB)

 Trainable params: 19,620 (76.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5900 - loss: 1.2510 - val_accuracy: 0.6500 - val_loss: 1.0767
Epoch 2/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6827 - loss: 1.0305 - val_accuracy: 0.6500 - val_loss: 1.0099
Epoch 3/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7138 - loss: 0.9354 - val_accuracy: 0.6500 - val_loss: 1.0229
Epoch 4/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6699 - loss: 1.0156 - val_accuracy: 0.6500 - val_loss: 1.0154
Epoch 5/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6907 - loss: 0.9714 - val_accuracy: 0.6500 - val_loss: 1.0175
Epoch 6/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6858 - loss: 0.9863 - val_accuracy: 0.6500 - val_loss: 1.0034
Epoch 7/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7009 - loss: 0.9573 - val_accuracy: 0.6500 - val_loss: 0.9985
Epoch 8/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6893 - loss: 0.9708 - val_accuracy: 0.6500 - v

c:\Users\Haard\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Haard\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Haard\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo